In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# problem_lists = ['JOS1', 'AP1', 'AP2', 'AP4', 'FDS', 'FF1', 
# 'IKK1', 'BK1', 'DGO1', 'DGO2', 'FA1', 'FAR1', 'LOV1', 'MOP2', 'POLONI', 'SD', 'TOI4', 'TRIDIA', 'ZDT2']

problem_lists = ['AP1', 'AP2', 'AP4', 'DGO2', 'FDS', 'JOS1', 'LOV1', 'MOP2', 'POLONI', 
'SD', 'TOI4', 'TRIDIA']

In [4]:
admm_df = pd.read_csv('results_fact_3_admm_new/metrics.csv')
prox_df = pd.read_csv('results_prox_new/metrics.csv')

# FA1 values are NaN mostly
prox_df = prox_df[prox_df['problem'].apply(lambda x:False if 'FA1' in x else True)]
admm_df = admm_df[admm_df['problem'].apply(lambda x:False if 'FA1' in x else True)]
print(len(admm_df), len(prox_df))

admm_df = admm_df.sort_values(by='problem').reset_index(drop=True)
prox_df = prox_df.sort_values(by='problem').reset_index(drop=True)

def func(x):
    if x[-2]=='l':
        return x[:-2] + 'L1'
    else:
        return x

prox_df['problem'] = prox_df['problem'].apply(func)
print(len(admm_df), len(prox_df))

36 36
36 36


| **Metric**                               | **Better When** | **Explanation**                                                                         |
| ---------------------------------------- | --------------- | --------------------------------------------------------------------------------------- |
| **GD** (Generational Distance)           | 🔽 **Lower**    | Measures average distance to the true Pareto front. Closer = better.                    |
| **IGD** (Inverted Generational Distance) | 🔽 **Lower**    | Measures how well the Pareto front is covered. Lower = better coverage.                 |
| **Hypervolume**                          | 🔼 **Higher**   | Measures the volume dominated by the solution set. Larger = better.                     |
| **Spacing**                              | 🔽 **Lower**    | Measures distribution uniformity. Lower = more evenly spread solutions.                 |
| **Spread**                               | 🔽 **Lower**    | Measures extent and uniformity. Lower = better diversity and coverage.                  |
| **Epsilon** (Additive ε-indicator)       | 🔽 **Lower**    | Measures how much a set must be shifted to dominate a reference. Lower = better.        |
| **Coverage** (C-Metric)                  | 🔼 **Higher**   | Measures how much one set dominates another. Higher = better (if compared to baseline). |
| **R2**                                   | 🔼 **Higher**   | Scalarization-based indicator. Higher = better approximation to Pareto front.           |
| **Average Hausdorff**                    | 🔽 **Lower**    | Combines GD and IGD. Lower = closer and better coverage of Pareto front.                |
| **Error Ratio**                          | 🔽 **Lower**    | Fraction of solutions not on the true Pareto front. Lower = better.                     |


In [5]:
# def compute_performance_profile(metric_name, df1 = admm_df, df2 = prox_df, solver_names=('ADMM', 'Proximal'), eps=1e-9):
#     # Ensure same problem ordering and existence
#     assert all(df1.iloc[:, 0] == df2.iloc[:, 0]), "Problem names do not match or are not aligned."
#     problems = df1.iloc[:, 0].values
#     n_problems = len(problems)

#     # Extract the metric
#     metric1 = df1[metric_name].values + eps  # Add epsilon to avoid division by zero
#     metric2 = df2[metric_name].values + eps  # Add epsilon to avoid division by zero

#     # Stack into 2D array: rows are problems, columns are solvers
#     results = np.vstack([metric1, metric2]).T  # shape: (n_problems, 2)

#     # Compute performance ratios
#     best = np.min(results, axis=1)
#     ratios = results / best[:, np.newaxis]  # shape: (n_problems, 2)

#     # Define τ grid
#     taus = np.linspace(1, np.max(ratios)*1.05, 100)
    
#     # Compute ρ_s(τ)
#     rhos = []
#     for solver_idx in range(2):
#         rho_tau = [np.mean(ratios[:, solver_idx] <= tau) for tau in taus]
#         rhos.append(rho_tau)

#     # Plot
#     plt.figure(figsize=(8, 6))
#     for solver_idx, solver_name in enumerate(solver_names):
#         plt.plot(taus, rhos[solver_idx], label=solver_name)
#     plt.xlabel(r'$\tau$')
#     plt.ylabel(r'$\rho_s(\tau)$')
#     plt.title(f'Performance Profile for Metric: {metric_name}')
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()


In [6]:
def plot_performance_profile(metric, df1 = admm_df, df2 = prox_df, solver_names=('ADMM', 'Proximal'), eps=1e-5):
    # common_problems = set(df1.iloc[:, 0]) & set(df2.iloc[:, 0])
    # df1 = df1[df1.iloc[:, 0].isin(common_problems)].sort_values(by=df1.columns[0])
    # df2 = df2[df2.iloc[:, 0].isin(common_problems)].sort_values(by=df2.columns[0])

    # Metric column index
    try:
        col_idx_1 = df1.columns.get_loc(metric)
        col_idx_2 = df2.columns.get_loc(metric)
    except KeyError:
        raise ValueError(f"Metric '{metric}' not found in one of the files.")

    # Extract values
    values1 = df1.iloc[:, col_idx_1].values + eps  # Add epsilon to avoid division by zero
    values2 = df2.iloc[:, col_idx_2].values + eps  # Add epsilon to avoid division by zero
    results = np.vstack([values1, values2]).T  # shape: (n_problems, 2)

    # Metrics where higher is better
    higher_is_better = {
        'Hypervolume', 'Coverage', 'R2'
    }

    if metric in higher_is_better:
        # Flip ratio: max / t => lower is better
        best = np.max(results, axis=1)
        ratios = best[:, np.newaxis] / results
    else:
        # Normal: t / min => lower is better
        best = np.min(results, axis=1)
        ratios = results / best[:, np.newaxis]

    # Adaptive tau range
    tau_max = np.max(ratios) * 1.05
    taus = np.linspace(1, tau_max, 5000)

    # Compute performance profile
    rhos = []
    for solver_idx in range(len(solver_names)):
        rho_tau = [np.mean(ratios[:, solver_idx] <= tau) for tau in taus]
        rhos.append(rho_tau)

    return taus, rhos, solver_names


In [8]:
metric_names = [f for f in admm_df.columns if f not in {'problem', 'IGD', 'Epsilon', 'Coverage', 'R2'}]

# fig, axs = plt.subplots(4, 2, figsize=(12, 16))
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
for j, i in enumerate(metric_names[4:]):
    taus, rhos, solver_names = plot_performance_profile(i, admm_df, prox_df, solver_names=('ADMM', 'Proximal'))
    for idx in range(2):
        axs[j//2, j%2].plot(taus, rhos[idx], label=solver_names[idx], linewidth=2)
    
    axs[j//2, j%2].set_xlabel(r'$\tau$', fontsize=14)
    axs[j//2, j%2].set_ylabel(r'Performance profile $\rho_s(\tau)$', fontsize=14)
    if i=='avg_time':
        i = 'Average Time'
    elif i=='avg_iter':
        i = 'Average Iterations'
    elif i=='Average_Hausdorff':
        i = 'Average Hausdorff'
    elif i=='Error_Ratio':
        i = 'Error Ratio'
    axs[j//2, j%2].set_title(f'{i}', fontsize=15)
    axs[j//2, j%2].legend()
    axs[j//2, j%2].grid(False)


    # compute_performance_profile(i, admm_df, prox_df, solver_names=('ADMM', 'Proximal'), eps=1e-9)
plt.tight_layout()
# plt.savefig('performance_profiles.png', bbox_inches='tight')
plt.savefig('performance_profile_ppt2.png', bbox_inches='tight')
# plt.show()
plt.close()

In [8]:
# stichting the images together for a better comparison
import os
import matplotlib
import matplotlib.pyplot as plt
prox_imgs = [os.path.join('results_prox_new', f) for f in os.listdir('results_prox_new') if f.endswith('.png')]
prox_imgs = sorted(prox_imgs)
admm_imgs = [os.path.join('results_fact_3_admm_new', f) for f in os.listdir('results_fact_3_admm_new') if f.endswith('.png')]
admm_imgs = sorted(admm_imgs)


# probs_to_remove = ['IKK', 'FA', 'FF1', 'DGO1']
# for prob in probs_to_remove:
#     prox_imgs = [f for f in prox_imgs if prob not in f]
#     admm_imgs = [f for f in admm_imgs if prob not in f]


new_admm_imgs, new_prox_imgs = [], []
for prob in problem_lists:
    temp_a = [f for f in admm_imgs if prob in f]
    temp_b = [f for f in prox_imgs if prob in f]
    
    if len(temp_a) > 0 and len(temp_b) > 0:
        new_admm_imgs.append(temp_a)
        new_prox_imgs.append(temp_b)
#     admm_imgs = [f for f in admm_imgs if prob in f]
#     prox_imgs = [f for f in prox_imgs if prob in f]

# len(prox_imgs), len(admm_imgs)

# admm_imgs = np.concatenate(new_admm_imgs).tolist()
# prox_imgs = np.concatenate(new_prox_imgs).tolist()
# admm_imgs = sorted(admm_imgs)
# prox_imgs = sorted(prox_imgs)
admm_imgs = new_admm_imgs
prox_imgs = new_prox_imgs
print(len(admm_imgs), len(prox_imgs))

12 12


In [11]:
# save 3 images in one figure for comparison
# for prox_path, admm_path in zip(prox_imgs, admm_imgs):
num_imgs_single = 2
for i in range(0, len(admm_imgs), num_imgs_single):
    # print(f"i = {i}")
    # Create a new figure with two subplots
    fig, axs = plt.subplots(num_imgs_single, 2, figsize=(12, num_imgs_single * 4))
    for j in range(i, min(i + num_imgs_single, len(prox_imgs))):
        # print(f"j = {j}")
        # prox_img = plt.imread(prox_imgs[j])
        admm_img1 = plt.imread(admm_imgs[j][0])
        admm_img2 = plt.imread(admm_imgs[j][1])

        axs[j - i, 0].imshow(admm_img1)
        axs[j - i, 0].axis('off')
        # axs[j - i, 0].set_title(f'Proximal')

        axs[j - i, 1].imshow(admm_img2)
        axs[j - i, 1].axis('off')
            # axs[j - i, 1].set_title(f'ADMM')


        # axs[0].imshow(prox_img)
        # axs[0].axis('off')
        # axs[0].set_title('Proximal')

        # axs[1].imshow(admm_img)
        # axs[1].axis('off')
        # axs[1].set_title('ADMM')

    plt.tight_layout()
    plt.savefig(f'admm_results_ppt_{i//num_imgs_single}.png', dpi=300, bbox_inches='tight')
    plt.close()
    # plt.show()

In [19]:
# admm_df = admm_df[~admm_df['problem'].str.contains('|'.join(probs_to_remove))].reset_index(drop=True)
# prox_df = prox_df[~prox_df['problem'].str.contains('|'.join(probs_to_remove))].reset_index(drop=True)
admm_df = admm_df[admm_df['problem'].str.contains('|'.join(problem_lists))].reset_index(drop=True)
prox_df = prox_df[prox_df['problem'].str.contains('|'.join(problem_lists))].reset_index(drop=True)


# cols_to_select = ['Problem', 'g(x)'] +  list(admm_df.columns[1:])
cols_to_select = ['Problem', 'g(x)'] + metric_names + ['R2', 'Coverage']

admm_df['Problem'] = admm_df['problem'].apply(lambda x:x.split(',')[0])
prox_df['Problem'] = prox_df['problem'].apply(lambda x:x.split(',')[0])

admm_df['g(x)'] = admm_df['problem'].apply(lambda x:'l1' if x[-2:]=='L1' else '0')
prox_df['g(x)'] = prox_df['problem'].apply(lambda x:'l1' if x[-2:]=='L1' else '0')


# admm_df.loc[:, cols_to_select].to_csv('final_admm_results_table.csv', index=False)
# prox_df.loc[:, cols_to_select].to_csv('final_prox_results_table.csv', index=False)
problems_info = pd.read_csv('/home/vm270/ayush/expts/Multi-Objective-Optimization/methods/Final_ADMM/results_new/problems_info.csv')
admm_df.merge(problems_info, on='Problem', how='left').loc[:, cols_to_select[:1] + ['m', 'n'] + cols_to_select[1:]].to_csv('final_admm_results_table.csv', index=False)
prox_df.merge(problems_info, on='Problem', how='left').loc[:, cols_to_select[:1] + ['m', 'n'] + cols_to_select[1:]].to_csv('final_prox_results_table.csv', index=False)